In [199]:
import pandas as pd
import numpy as np

# 1. Dataset

(1) tag dataset

In [16]:
# tag preprocessing
def tag(df):
    tags = df["tags"].apply(lambda x: x.split(','))
    tags = tags.apply(lambda x: list(map(lambda x: x.replace("[", "").replace("]", "").replace("'", "").strip(), x)))
    tag_ls = []
    for tag in tags:
        tag_ls.extend(tag)
    return tag_ls    

In [17]:
insta = tag(insta)
youtube = tag(youtube)
tags = insta+youtube

(2) Sales dataset

In [181]:
sales = pd.read_csv("sales.csv")

# 2. Korean Analysis with cosine-similarity

(1) match tag-sales

In [119]:
kr_pd_nm = pd.unique(sale["**special-code**"])
kr_pd_nm = kr_pd_nm[~pd.isnull(kr_pd_nm)]

In [6]:
def cleanText(text):
    import re
    text = re.sub('[0-9-=+,#/\?:^$.@*\\\\"~&(\)\[\]\<\>"]','',text)
    return text

In [8]:
#calculate distance between two words
def calc_distance(a,b):
    if a==b : return 0
    a_len=len(a)
    b_len=len(b)
    if a=="" : return b_len 
    if b=="" : return a_len

    matrix= [ []for i in range(a_len+1)]
    for i in range(a_len+1) :
        matrix[i] = [0 for j in range(b_len+1)]
    for i in range(a_len+1):
        matrix[i][0]=i   
    for j in range(b_len+1):
        matrix[0][j]=j
    for i in range(1,a_len+1):
        ac=a[i-1]  
        for j in range(1, b_len+1):
            bc=b[j-1] 
            cost=0 if (ac ==bc) else 1 
            matrix[i][j] = min([  
                matrix[i-1][j] + 1,
                matrix[i][j-1] + 1,
                matrix[i-1][j-1] + cost
            ])
    return matrix[a_len][b_len]

In [142]:
def lev_cal_words(word, product_list, value):
    import unicodedata
    import math
    min_rate = 100
    min_product = []
    word1 = word.replace(" ","")
    word1 = unicodedata.normalize("NFC",word1)
    for product in product_list:
        word2 = product.replace(" ", "")
        distance_rate = math.floor(((calc_distance(word1, word2))/(len(word2)))*10)
        if  min_rate > distance_rate:
            min_product = []
            min_product.append(product)
            min_rate = distance_rate
        elif min_rate == distance_rate:
            min_product.append(product)
    if min_rate >= value:                #value depends on the language        
        min_product = None
    return min_product

In [148]:
def kr_tag_sales(tags, kr_pd_nm):
    kr_tags = []; kr_products = []
    for tag in tags:
        value = lev_cal_words(tag, kr_pd_nm, 7)
        if value != None:
            for product in value:
                kr_tags.append(tag)
                kr_products.append(product)
    df = pd.DataFrame(zip(kr_tags, kr_products), columns = ["tags", "pd_nm"])
    return df

In [219]:
kr_pd_nm = pd.unique(sales["**special_code**"])
kr_df = kr_tag_sales(tags, kr_pd_nm)
kr_df = kr_df.groupby("pd_nm").count().rename_axis("pd_nm").reset_index()

(2) match tag-sales count

In [224]:
prod_amt = sales.groupby("PRIMARY_PROD_NM").sum().rename_axis("PRIMARY_PROD_NM").reset_index()
kr_tag_sales = pd.merge(kr_df, prod_amt, left_on = "pd_nm", right_on = "PRIMARY_PROD_NM").drop("PRIMARY_PROD_NM", axis=1)
kr_tag_sales = kr_tag_sales.rename(columns = {"tags":"tag_count", "pd_nm": "product", "PROD_BY_AMT" : "sales_amt"})

In [226]:
kr_tag_sales["log_tag_count"] = kr_tag_sales["tag_count"].apply(lambda x: np.log1p(x))
kr_tag_sales["log_sales_amt"] = kr_tag_sales["sales_amt"].apply(lambda x: np.log1p(x))
kr_tag_sales = kr_tag_sales.drop(["tag_count", "sales_amt"], axis=1)